#### 1. Generate raw dataset without OOS
#### 2. Generate summary table(Normality stats, original Ppk)
#### 3. Prepare the subset of raw dataset containing several parameters for JMP 

In [1]:
import pandas as pd
import numpy as np

import os

from scipy.stats import anderson
from scipy.stats import kstest
from scipy.stats import shapiro

def kurt(values):
    n=len(values)
    if (n<4): 
        result=np.nan
    else:
        sample_mean=np.mean(values)
        sample_std=np.std(values,ddof=1)
        result=np.sum(((values-sample_mean)/sample_std)**4)*n*(n+1)/((n-1)*(n-2)*(n-3))-3*(n-1)**2/((n-2)*(n-3))
    return(result)

def skewness(values):
    n=len(values)
    if (n<3):
        result=np.nan
    else:
        sample_mean=np.mean(values)
        sample_std=np.std(values,ddof=1)
        result=n*np.sum(((values-sample_mean)/sample_std)**3)/((n-1)*(n-2))
    return(result)

def CV(values):
    sample_mean=np.mean(values)
    sample_std=np.std(values,ddof=1)
    return(100*sample_std/sample_mean)

def ppk_single_row(values,ual,lal,spec_status):
    if spec_status==0:
        ppk=np.minimum(float(ual)-np.mean(values),np.mean(values)-float(lal) )/(3*np.std(values,ddof=1))
    else:
        if spec_status==1:
            ppk=(float(ual)-np.mean(values))/(3*np.std(values,ddof=1))
        else:
            if spec_status==-1:
                ppk=(np.mean(values)-float(lal))/(3*np.std(values,ddof=1))
            else:
                ppk="NA3"
    return(ppk)

def add_specside(df):
    #no spec default
    df["spec status"]=None
    #two specs
    df["spec status"].loc[(df.UAL!=".") & (df.LAL!=".")]=0
    #only lower spec
    df["spec status"].loc[(df.UAL==".") & (df.LAL!=".")]=-1
    #only upper spec
    df["spec status"].loc[(df.UAL!=".") & (df.LAL==".")]=1
    
def add_clside(df):
    #no spec default
    df["ctrl status"]=None
    #two specs
    df["ctrl status"].loc[(df.LCL!=".") & (df.UCL!=".")]=0
    #only lower spec
    df["ctrl status"].loc[(df.UCL==".") & (df.LCL!=".")]=-1
    #only upper spec
    df["ctrl status"].loc[(df.UCL!=".") & (df.LCL==".")]=1

def df_colname_unify(df):
    df=df.rename(columns={"ual":"UAL","lal":"LAL","lcl":"LCL","ucl":"UCL"})

def quanti_quali_namelists(df):
    
    qualitative_paras=list(pd.unique(df["parameter name"][df['result type']=="TEXT"]))

    quantitative_paras=list(pd.unique(df["parameter name"][df['result type']=="Number"]))
    
    return(quantitative_paras,qualitative_paras)

def para_unit_combi_df(df):
    para_unit_combi=df[["parameter name","unit procedure"]].copy().drop_duplicates().reset_index(drop=True)
    para_unit_combi=para_unit_combi.sort_values(by=["unit procedure","parameter name"])
    return(para_unit_combi)

#only use a quantitative dataset as the input
def spec_correction_rawset(df,para_unit_pair_df):
    for i in range(len(para_unit_pair_df)):
        para=para_unit_pair_df["parameter name"].iloc[i]
        unit=para_unit_pair_df["unit procedure"].iloc[i]
        
        sub_df_len=len(df.loc[(df['parameter name']==para) &(df["unit procedure"]==unit)])
        
        #more than one row, needs spec correction
        if (sub_df_len>1):
           
            #could loop using the uniq combination of unit and para, otherwise add this row number check to avoid void error
            ual=df[(df['parameter name']==para) &(df["unit procedure"]==unit)].UAL.iloc[-1]
        
            lal=df[(df['parameter name']==para) &(df["unit procedure"]==unit)].LAL.iloc[-1]
            df["UAL"].loc[(df['parameter name']==para) &(df["unit procedure"]==unit)]=ual
            df["LAL"].loc[(df['parameter name']==para) &(df["unit procedure"]==unit)]=lal

def find_oos_index(df):
    df_twospecs=df[df["spec status"]==0]
    df_uspecs=df[df["spec status"]==1]
    df_lspecs=df[df["spec status"]==-1]
    
    #Find the indexes of OOS rows, try to do this in SQL
    twospecs_rows_oos=list(df_twospecs.index[(df_twospecs["value_num(recorded/full precision)"]<df_twospecs.LAL)\
                    |(df_twospecs["value_num(recorded/full precision)"]>df_twospecs.UAL)])

    uspecs_rows_oos=list(df_uspecs.index[(df_uspecs["value_num(recorded/full precision)"]>df_uspecs.UAL)])

    lspecs_rows_oos=list(df_lspecs.index[(df_lspecs["value_num(recorded/full precision)"]<df_lspecs.LAL)])
    
    return(list(set(twospecs_rows_oos+uspecs_rows_oos+lspecs_rows_oos)))

def find_oot_index(df):
    df_twocls=df[df["ctrl status"]==0]
    df_ucl=df[df["ctrl status"]==1]
    df_lcl=df[df["ctrl status"]==-1]
    
    twocls_rows_oot=list(df_twocls.index[(df_twocls["value_num(recorded/full precision)"]<df_twocls.LCL)\
                |(df_twocls["value_num(recorded/full precision)"]>df_twocls.UCL)])

    ucls_rows_oot=list(df_ucl.index[(df_ucl["value_num(recorded/full precision)"]>df_ucl.UCL)])

    lcls_rows_oot=list(df_lcl.index[(df_lcl["value_num(recorded/full precision)"]<df_lcl.LCL)])

    return(list(set(twocls_rows_oot+ucls_rows_oot+lcls_rows_oot)))

#change the threshold(30) if we allow smaller sample size after excluding the OOS
#input_type can be "" or "_no_OOS" or "no_OOST", etc, used to define the the column name
def aggre_summary_df_generator(df, para_unit_pair_df, input_type):
    
    #initialize the summary output df
    aggre_stats_output_df=pd.DataFrame(columns=["parameter name",'unit procedure',#"PRDS",#\
                                           "N","mean","median","STD","variance","UAL","LAL","spec status","Ppk"+input_type,\
                                           "skewness","kurtosis","CV",#"anderson-darling"\
                                            "shapiro","KS"\
                                              ])
    for i in range(len(para_unit_pair_df)):
        
        para=para_unit_pair_df["parameter name"].iloc[i]
        unit=para_unit_pair_df["unit procedure"].iloc[i]
        
        single_df=df.loc[(df['parameter name']==para) &(df["unit procedure"]==unit)].copy().sort_values(by="parameter date") 
        single_df_values=single_df["value_num(recorded/full precision)"]  
        
        ual=single_df.UAL.iloc[-1]
        lal=single_df.LAL.iloc[-1]
        spec_status=single_df["spec status"].iloc[-1]

        if (len(single_df)<30):
            ppk="NA1"

        if( (len(single_df)>=30) and (len(pd.unique(single_df_values)<5)) ):
            ppk="NA2"

        #NA3: no specs is addressed inside the ppk_single_row
           
        if ((len(single_df)>=30) and (len(pd.unique(single_df_values)>=5)) ):
 
            #prds=pd.unique(single_df["parameter detail"])
            
            ppk=ppk_single_row(single_df_values,ual,lal,spec_status)
        if (len(single_df)<4):
            shap=np.nan
            ks=np.nan
        else:
            shap=shapiro(single_df_values)[1]
            ks=kstest(single_df_values,"norm")[1]
        
        aggre_stats_output_df=aggre_stats_output_df.append({\
                        'parameter name':para,\
                        "unit procedure":unit,\
                        #"PRDS":prds,
                        "N":len(single_df_values),\
                        "mean":np.mean(single_df_values),\
                        "STD":np.std(single_df_values,ddof=1),\
                        "variance":np.var(single_df_values,ddof=1),\
                        "median":np.median(single_df_values),\
                        "LAL":lal, "UAL":ual,      \
                        "spec status":spec_status ,\
                        "Ppk"+input_type: ppk,\
                        "skewness":skewness(single_df_values),\
                        "kurtosis":kurt(single_df_values),\
                        "CV":CV(single_df_values),\
                        #"Anderson-Darling":anderson(sub_df),\
                        "shapiro":shap,\
                        #the p value of shapiro
                       "KS":ks},\
                        #the p value of kstest
                                    ignore_index=True)
    return(aggre_stats_output_df)

def JMP_single_para_subset_generator(para_unit_pair_df, raw_df, folder_path):
    os.chdir(folder_path)
    for i in range(len(para_unit_pair_df)):
        para=para_unit_pair_df["parameter name"].iloc[i]
        unit=para_unit_pair_df["unit procedure"].iloc[i]
        single_subset=raw_df[(raw_df["parameter name"]==para)&(raw_df["unit procedure"]==unit)].copy()
        single_subset.to_csv(para+"-"+unit+"JMP.csv", index=False)
    
    return

In [135]:
aggre_summary_df_generator(test, para_unit_pair_df,"_no_OOS")

,parameter name,unit procedure,N,mean,median,STD,variance,UAL,LAL,spec status,Ppk_no_OOS,skewness,kurtosis,CV,shapiro,KS
0,Chloride,IEDL:ADL9-DP-ADL,50,138.662200,138.485000,1.696981,2.879744e+00,170,110,0.0,5.63004,1.105692,3.662593,1.223824,2.289855e-03,0.000000e+00
1,Immunoassay,IEDL:ADL9-DP-ADL,57,1.031193,1.023500,0.053303,2.841241e-03,1.2,0.8,0.0,1.05564,0.331360,-0.327343,5.169090,7.188891e-01,0.000000e+00
2,In vitro bioassay,IEDL:ADL9-DP-ADL,50,104.400000,104.500000,6.077728,3.693878e+01,123,78,0.0,1.02012,-0.041020,1.787548,5.821578,8.635213e-02,0.000000e+00
3,Methionine Oxidation,IEDL:ADL9-DP-ADL,50,1.321600,1.325000,0.147196,2.166678e-02,4,.,1.0,6.06537,-0.067031,-0.043242,11.137741,9.741972e-01,0.000000e+00
4,Phosphate,IEDL:ADL9-DP-ADL,50,19.904980,19.885500,0.265579,7.053206e-02,24,16,0.0,4.90122,0.299120,0.480663,1.334233,6.419759e-01,0.000000e+00
5,Polysorbate 80,IEDL:ADL9-DP-ADL,57,0.004544,0.004500,0.000348,1.210777e-07,0.007,0.003,0.0,1.47895,0.070300,-0.644668,7.657855,3.491033e-01,8.637535e-14
6,SE-HPLC,IEDL:ADL9-DP-ADL,50,99.972880,99.973000,0.002819,7.944490e-06,.,99.5,-1.0,55.9238,-0.134594,0.368606,0.002819,2.752670e-01,0.000000e+00
7,Subvisible Particles >= 10 um,IEDL:ADL9-DP-ADL,50,109.054000,42.810000,151.458606,2.293971e+04,6000,.,1.0,12.9649,2.959013,10.058229,138.884045,2.261463e-10,0.000000e+00
8,Subvisible Particles >= 25 um,IEDL:ADL9-DP-ADL,50,1.175400,0.370000,1.820657,3.314793e+00,600,.,1.0,109.635,2.902401,9.641608,154.896819,3.174999e-10,1.106670e-12
9,Volume (0.3 mL Syringe),IEDL:ADL9-DP-ADL,15,0.332540,0.332500,0.002326,5.412571e-06,.,0.3,-1.0,NA1,0.669534,0.049823,0.699613,3.923467e-01,2.825563e-06


In [141]:
pd.read_csv("aranesp_no_oost_Oct22.csv",index_col=False)

,Unnamed: 0,unit procedure,sampling point,parameter name,parameter detail,run number,batch number,date of manufacture,process start,parameter date,...,scale,summary table,disposition_date,result type,source_system,ctrl_violation,action_violation,rej_violation,spec status,ctrl status
0,100,IEDL:ADL9-DP-ADL,.,Chloride,PRDS-000444,10340208,10340208,2017-07-11,2017-07-09,2017-07-11,...,LIN,.,.,Number,SMLIMS,.,.,.,0.0,NaN
1,101,IEDL:ADL9-DP-ADL,.,Chloride,PRDS-000443,10340209,10340209,2017-07-13,2017-07-10,2017-07-13,...,LIN,.,.,Number,SMLIMS,.,.,.,0.0,NaN
2,102,IEDL:ADL9-DP-ADL,.,Chloride,PRDS-000443,10344080,10344080,2017-07-18,2017-07-18,2017-07-18,...,LIN,.,.,Number,SMLIMS,.,.,.,0.0,NaN
3,103,IEDL:ADL9-DP-ADL,.,Chloride,PRDS-000444,10340210,10340210,2017-07-19,2017-07-17,2017-07-19,...,LIN,.,.,Number,SMLIMS,.,.,.,0.0,NaN
4,104,IEDL:ADL9-DP-ADL,.,Chloride,PRDS-000443,10340211,10340211,2017-07-25,2017-07-17,2017-07-25,...,LIN,.,.,Number,SMLIMS,.,.,.,0.0,NaN
5,105,IEDL:ADL9-DP-ADL,.,Chloride,PRDS-000449,10345848,10345848,2017-07-31,2017-07-31,2017-07-31,...,LIN,.,.,Number,SMLIMS,.,.,.,0.0,NaN
6,106,IEDL:ADL9-DP-ADL,.,Chloride,PRDS-000449,10344209,10344209,2017-08-02,2017-07-27,2017-08-02,...,LIN,.,.,Number,SMLIMS,.,.,.,0.0,NaN
7,107,IEDL:ADL9-DP-ADL,.,Chloride,PRDS-000444,10349699,10349699,2017-09-14,2017-09-13,2017-09-14,...,LIN,.,.,Number,SMLIMS,.,.,.,0.0,NaN
8,108,IEDL:ADL9-DP-ADL,.,Chloride,PRDS-000443,10350648,10350648,2017-09-25,2017-09-23,2017-09-25,...,LIN,.,.,Number,SMLIMS,.,.,.,0.0,NaN
9,109,IEDL:ADL9-DP-ADL,.,Chloride,PRDS-000443,10354042,10354042,2017-10-02,2017-10-02,2017-10-02,...,LIN,.,.,Number,SMLIMS,.,.,.,0.0,NaN


In [136]:
test

,Unnamed: 0,unit procedure,sampling point,parameter name,parameter detail,run number,batch number,date of manufacture,process start,parameter date,...,scale,summary table,disposition_date,result type,source_system,ctrl_violation,action_violation,rej_violation,spec status,ctrl status
0,100,IEDL:ADL9-DP-ADL,.,Chloride,PRDS-000444,10340208,10340208,2017-07-11,2017-07-09,2017-07-11,...,LIN,.,.,Number,SMLIMS,.,.,.,0.0,NaN
1,101,IEDL:ADL9-DP-ADL,.,Chloride,PRDS-000443,10340209,10340209,2017-07-13,2017-07-10,2017-07-13,...,LIN,.,.,Number,SMLIMS,.,.,.,0.0,NaN
2,102,IEDL:ADL9-DP-ADL,.,Chloride,PRDS-000443,10344080,10344080,2017-07-18,2017-07-18,2017-07-18,...,LIN,.,.,Number,SMLIMS,.,.,.,0.0,NaN
3,103,IEDL:ADL9-DP-ADL,.,Chloride,PRDS-000444,10340210,10340210,2017-07-19,2017-07-17,2017-07-19,...,LIN,.,.,Number,SMLIMS,.,.,.,0.0,NaN
4,104,IEDL:ADL9-DP-ADL,.,Chloride,PRDS-000443,10340211,10340211,2017-07-25,2017-07-17,2017-07-25,...,LIN,.,.,Number,SMLIMS,.,.,.,0.0,NaN
5,105,IEDL:ADL9-DP-ADL,.,Chloride,PRDS-000449,10345848,10345848,2017-07-31,2017-07-31,2017-07-31,...,LIN,.,.,Number,SMLIMS,.,.,.,0.0,NaN
6,106,IEDL:ADL9-DP-ADL,.,Chloride,PRDS-000449,10344209,10344209,2017-08-02,2017-07-27,2017-08-02,...,LIN,.,.,Number,SMLIMS,.,.,.,0.0,NaN
7,107,IEDL:ADL9-DP-ADL,.,Chloride,PRDS-000444,10349699,10349699,2017-09-14,2017-09-13,2017-09-14,...,LIN,.,.,Number,SMLIMS,.,.,.,0.0,NaN
8,108,IEDL:ADL9-DP-ADL,.,Chloride,PRDS-000443,10350648,10350648,2017-09-25,2017-09-23,2017-09-25,...,LIN,.,.,Number,SMLIMS,.,.,.,0.0,NaN
9,109,IEDL:ADL9-DP-ADL,.,Chloride,PRDS-000443,10354042,10354042,2017-10-02,2017-10-02,2017-10-02,...,LIN,.,.,Number,SMLIMS,.,.,.,0.0,NaN


In [101]:
immu=pd.read_csv("normal_stats_output_all_Oct11.csv")

In [66]:
test["LAL"]=pd.to_numeric(test.LAL,errors="ignore")
test["UAL"]=pd.to_numeric(test.UAL,errors="ignore")

In [7]:
aranesp=pd.read_excel("query-impala-154041.xlsx")

In [131]:
para_unit_pair_df=para_unit_combi_df(test)

In [12]:
aranesp=aranesp.rename(columns={"ual":"UAL","lal":"LAL","lcl":"LCL","ucl":"UCL"})

In [16]:
spec_correction_rawset(aranesp,para_unit_pair_df)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [18]:
 aranesp[(aranesp["parameter name"]=="Isoelectric Focusing: Isoforms 19 + 20")&(aranesp["unit procedure"]=="PRJU:APR6-DS-AML")]["UAL"]

5610    43
5611    43
5612    43
5613    43
5614    43
5615    43
5616    43
5617    43
5618    43
5619    43
5620    43
5621    43
5622    43
5623    43
5624    43
5625    43
5626    43
5627    43
5628    43
5629    43
5630    43
5631    43
5632    43
5633    43
5634    43
5635    43
5636    43
5637    43
5638    43
5639    43
Name: UAL, dtype: object

In [5]:
tttt()[0]
df1.groupby(['A','B']).size()

1